# Imports

In [7]:
## Import necessary libraries here (You can add libraries you want to use here)
import os
import random
import cv2
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
from google.colab.patches import cv2_imshow
import itertools
import copy
from math import sqrt
import math
from scipy.linalg import sqrtm
import plotly.graph_objects as go

# Part 1: Epipolar Geometry (30 Points)


## Overview

In this problem, you will implement an algorithm for automatically estimating homography with RANSAC. In the file matches.mat, we provide the detected Harris corners row-column positions in variables r1 c1 for the first image; variables r2 c2 for the second image; and the corresponding matched pairs in the variable matches.

<!-- <img src="https://drive.google.com/uc?id=1Tr723u5OXmwkd4RDmu9z886ITJU9j1cL&export=download" width="800"/> -->

<img src="https://drive.google.com/uc?id=17mwO8QH24vw1Kv1aBONgFXKi53HqUMEd&export=download" width="800"/>


The outline of the normalized 8-point algorithm:

<img src="https://drive.google.com/uc?id=1nVnvBpKeLmiowT9Q4_QauogXpcdXBmHm&export=download" width="700"/>



### Data

**WARNING: Colab deletes all files everytime runtime is disconnected. Make sure to re-download the inputs when it happens.**

In [36]:
# Download Data -- run this cell only one time per runtime
!gdown 1cn3_SscjlLrf4BzUWe8MV-XqMqBY4Nj_
!unzip "/content/Part1_data.zip" -d "/content/"
# Load Matches
data = loadmat('/content/Part1_data/matches.mat')
r1 = data['r1']
r2 = data['r2']
c1 = data['c1']
c2 = data['c2']
matches = data['matches']

Downloading...
From: https://drive.google.com/uc?id=1cn3_SscjlLrf4BzUWe8MV-XqMqBY4Nj_
To: /content/Part1_data.zip

  0% 0.00/157k [00:00<?, ?B/s]
100% 157k/157k [00:00<00:00, 77.2MB/s]
Archive:  /content/Part1_data.zip
   creating: /content/Part1_data/
  inflating: /content/Part1_data/chapel00.png  
  inflating: /content/Part1_data/chapel01.png  
  inflating: /content/Part1_data/matches.mat  


In [37]:
# Load Keypoints
x1 = c1[matches[:,0]-1]
y1 = r1[matches[:,0]-1]
x2 = c2[matches[:,1]-1]
y2 = r2[matches[:,1]-1]

### Code (15 pt)

In [ ]:
def normalize(x, y):
  # Function: find the transformation to make it zero mean and the variance as sqrt(2)
  std_x = np.std(x)
  std_y = np.std(y)
  mean_x = np.mean(x)
  mean_y = np.mean(y)
  T = np.array([[1/std_x,     0  , -mean_x/std_x],
                [    0  , 1/std_y, -mean_y/std_y],
                [    0  ,     0  ,      1       ]])
  normalized_coords = []
  for i in range(x.shape[0]):
    temp = np.matmul(T, np.array([x[i], y[i], 1], dtype=object))
    normalized_coords.append([temp[0][0], temp[1][0], temp[2][0]])
  normalized_coords = np.array(normalized_coords)
  normalized_x = normalized_coords[:, 0]
  normalized_y = normalized_coords[:, 1]
  return normalized_x, normalized_y, T
  
def ransacF(x1, y1, x2, y2, do_normalization=True, T1=None, T2=None):
  if do_normalization:
    # Find normalization matrix & Transform point set 1 and 2
    x1, y1, T1 = normalize(x1, y1)
    x2, y2, T2 = normalize(x2, y2)
  # RANSAC based 8-point algorithm
  threshold = 0.1
  max_num_inliers = -1
  num_iters = 0
  rand_indices = [i for i in range(x1.shape[0])]
  random.shuffle(rand_indices)
  for subset in itertools.combinations(rand_indices, 8):
    num_iters += 1
    if num_iters>1400:
      break
    x1_subset = x1[tuple([subset])]
    y1_subset = y1[tuple([subset])]
    x2_subset = x2[tuple([subset])]
    y2_subset = y2[tuple([subset])]
    F = computeF(x1_subset,y1_subset,x2_subset,y2_subset)
    num_inliers, distances = getInliers(x1, y1, x2, y2, F, threshold)
    if num_inliers > max_num_inliers:
      inlier_indices = tuple(np.arange(x1.shape[0])[distances==-1])
      max_num_inliers = num_inliers
      max_inlier_comb = inlier_indices 
  
  print('Number of iterations =', num_iters)
  print('Number of inliers =', max_num_inliers)
  x1_subset = x1[tuple([max_inlier_comb])]
  y1_subset = y1[tuple([max_inlier_comb])]
  x2_subset = x2[tuple([max_inlier_comb])]
  y2_subset = y2[tuple([max_inlier_comb])]
  F_norm = computeF(x1_subset,y1_subset,x2_subset,y2_subset)
  F = np.matmul(T1.T, np.matmul(F_norm, T2))
  return F, max_inlier_comb

def getInliers(x1, y1, x2, y2, F, threshold):
  # Function: implement the criteria checking inliers.
  distances = []
  for j in range(x1.shape[0]):
    line = np.matmul(F, np.array([x2[j], y2[j], 1]))
    temp = abs(np.matmul(np.array([x1[j], y1[j], 1]), line)) / sqrt(line[0]**2 + line[1]**2)
    distances.append(temp)
  distances = np.array(distances)
  assert np.amin(distances) >= 0, str(np.amin(distances))+' is the min number'
  distances[distances<=threshold] = -1
  distances[distances>threshold] = 0
  num_inliers = np.sum(distances) / -1
  return num_inliers, distances

  
def computeF(x1, y1, x2, y2):
  #  Function: compute fundamental matrix from corresponding points
  A = []
  for i in range(x1.shape[0]):
    A.append([x1[i]*x2[i], x1[i]*y2[i], x1[i], y1[i]*x2[i], y1[i]*y2[i], y1[i], x2[i], y2[i], 1])
  U, S, VT = np.linalg.svd(A)
  f = VT[-1]
  F = np.reshape(f, (3,3))
  u, s, vt = np.linalg.svd(F)
  s[-1] = 0
  s = np.diag(s)
  F = np.matmul(u, np.matmul(s, vt))
  return F

def gen_epi_line_and_plot(epi_line, x, y, img):
  rows, cols, _ = img.shape
  start1 = (0, int(-epi_line[2][0]/epi_line[1][0]))
  x_ = cols-1
  y_ = int(((-epi_line[0][0]*x_)-epi_line[2][0]) / epi_line[1][0])
  end1 = (x_, y_)
  img = cv2.line(img, start1, end1, (0, 255, 0), 1)
  img = cv2.circle(img, (int(x), int(y)), 2, (0, 0, 255), -1)
  return img

def plot_epi_lines(epi_line_1, epi_line_2, x_1, y_1, x_2, y_2, im1, im2, show_result):
  im1 = gen_epi_line_and_plot(epi_line_1, x_1, y_1, im1)
  im2 = gen_epi_line_and_plot(epi_line_2, x_2, y_2, im2)
  if show_result:
    combined_img = np.hstack((im1, im2))
    cv2_imshow(combined_img)

img1 = cv2.imread('/content/Part1_data/chapel00.png')
img2 = cv2.imread('/content/Part1_data/chapel01.png')
F, max_inlier_comb = ransacF(x1, y1, x2, y2)
F_unit = F / np.sqrt(np.sum(np.square(F)))
print(F_unit)
rand_7_indices=[]
while np.unique(rand_7_indices).shape[0] != 7:
  rand_7_indices = (np.random.rand(7)*x1.shape[0]).astype(int)

for rand_idx in rand_7_indices:
  x_1 = x1[rand_idx]
  y_1 = y1[rand_idx]
  x_2 = x2[rand_idx]
  y_2 = y2[rand_idx]
  epi_line_1 = np.matmul(F, np.array([x_2, y_2, 1], dtype=object))
  epi_line_2 = np.matmul(F.T, np.array([x_1, y_1, 1], dtype=object))
  plot_epi_lines(epi_line_1, epi_line_2, x_1, y_1, x_2, y_2, img1, img2, rand_idx==rand_7_indices[-1])

### Write-up (15 pt)
*   Describe what test you used for deciding inlier vs. outlier.
*   Display the estimated fundamental matrix F after normalizing to unit length
*   Randomly select 7 sets of matching points. Plot the corresponding epipolar lines and the points on each image. Show the two images (with plotted points and lines) next to each other.

<!-- *   Plot the outlier keypoints with green dots on top of the first image -->
<!-- *   Randomly select 7 sets of matching points. Plot the corresponding epipolar lines ('g’) and the points (with 'r+’) on each image. Show the two images (with plotted points and lines) next to each other. -->



- 1) Given a pair of matches x1 & x2, I computed the epipolar line of x2 in image1 & found the distance of the point x1 from the above computed epipolar line in img1 and thresholded it to decide if the pair of matches is an outlier.

- 2) Fundamental Matrix:

  <img src="https://drive.google.com/uc?id=1OdJPrN4R5N9Q_Kdk-OBtv_cAnbt_EkKb" width="400"/>

- 3) Epipolar Lines & associated points:

  <img src="https://drive.google.com/uc?id=1BLWXQE0PlyWIvBNsqK_qhNRYkK9h7YSq" width="800"/>

# Part 2: Image stitching (30 points)

<img src="https://drive.google.com/uc?id=1uOI8rpqb_FsR9Fi8GrGPZvICOcgflBj9&export=download" width="800"/>

## Overview

In this problem, you will implement an algorithm for automatically estimating the fundamental matrix F using RANSAC and the normalized 8-point algorithm. 

Image Stitching Algorithm Overview
1. Detect keypoints
2. Match keypoints
3. Estimate homography with matched keypoints (using RANSAC)
4. Combine images

**Note:**  Do not use existing image stitching code, such as found on the web, and OpenCV.

## Data

**WARNING: Colab deletes all files everytime runtime is disconnected. Make sure to re-download the inputs when it happens.**

In [41]:
# Download Data -- run this cell only one time per runtime
!gdown 1fnD0hJ8-_Rngsc-m96ghKtdZAMf0VTjy
!unzip "/content/hill.zip" -d "/content/hill"

!gdown 1v2BFVMV0McuD5BstLvDmo1U9MrFAByS5
!unzip "/content/tv.zip" -d "/content/tv"


Downloading...
From: https://drive.google.com/uc?id=1fnD0hJ8-_Rngsc-m96ghKtdZAMf0VTjy
To: /content/hill.zip

  0% 0.00/205k [00:00<?, ?B/s]
100% 205k/205k [00:00<00:00, 90.1MB/s]
Archive:  /content/hill.zip
  inflating: /content/hill/1.JPG     
  inflating: /content/hill/2.JPG     
  inflating: /content/hill/3.JPG     
Downloading...
From: https://drive.google.com/uc?id=1v2BFVMV0McuD5BstLvDmo1U9MrFAByS5
To: /content/tv.zip
100% 130k/130k [00:00<00:00, 71.9MB/s]
Archive:  /content/tv.zip
  inflating: /content/tv/1.jpg       
  inflating: /content/tv/2.jpg       
  inflating: /content/tv/3.jpg       


## Helper Functions

In [42]:
def plot_matches(img1, img2, keypoints_1, keypoints_2, matches):
  combined_img = np.hstack((img1, img2))
  img1_cols = img1.shape[1]
  colors = [(0, 0, 255), (0, 255,0), (255,0,0), (255,255,0), (0,255,255), (255,0,255)]
  j=0
  for i in range(len(matches)):
    pt1 = (round(tuple(keypoints_1[matches[i][0]].pt)[0])          , round(tuple(keypoints_1[matches[i][0]].pt)[1]))
    pt2 = (round(tuple(keypoints_2[matches[i][1]].pt)[0])+img1_cols, round(tuple(keypoints_2[matches[i][1]].pt)[1]))
    j %= len(colors)
    color = colors[j]
    j+=1
    combined_img = cv2.line(combined_img, pt1, pt2, color, 1)
  cv2_imshow(combined_img)

def est_homography(src, dest):
    N = src.shape[0]
    if N != dest.shape[0]:
        raise ValueError("src and diff should have the same dimension")
    src_h = np.hstack((src, np.ones((N, 1))))
    A = np.array([np.block([[src_h[n], np.zeros(3), -dest[n, 0] * src_h[n]],
                            [np.zeros(3), src_h[n], -dest[n, 1] * src_h[n]]])
                  for n in range(N)])
    A = A.reshape(2 * N, 9)
    [_, _, V] = np.linalg.svd(A)
    return V.T[:, 8].reshape(3, 3)

def apply_homography(H, src):
    src_h = np.hstack((src, np.ones((src.shape[0], 1))))
    dest =  src_h @ H.T
    if dest[:,[2]].any() == 0:
      print('divide by zero')
    return (dest / dest[:,[2]])[:,0:2]

### Code (15 pt)

In [ ]:
def compare(x):
  return x[0]

def compute_2nn_ratio(Descriptor1, Descriptor2):
  diff = Descriptor2 - Descriptor1
  dist = np.expand_dims(np.linalg.norm(diff, axis=1), axis=1)
  temp = np.expand_dims([t for t in range(dist.shape[0])], axis=1)
  dist = np.concatenate((dist, temp), axis=1).tolist()
  dist.sort(key=compare)
  return dist[0][0]/dist[1][0], int(dist[0][1])

def detect_keypoints_and_find_matches(img1, img2, display):
  sift = cv2.xfeatures2d.SIFT_create()
  keypoints_1, descriptors_1 = sift.detectAndCompute(img1,None)
  keypoints_2, descriptors_2 = sift.detectAndCompute(img2,None)
  matches = []
  threshold_2NN = 0.45 #0.45
  for i in range(len(keypoints_1)):
    ratio, closest_match = compute_2nn_ratio(descriptors_1[i], descriptors_2)
    if ratio < threshold_2NN:
      matches.append([i, closest_match])
  print('Number of matches =', len(matches))
  if display:
    plot_matches(img1, img2, keypoints_1, keypoints_2, matches)
  return matches, keypoints_1, keypoints_2

def normalize__(img1_shape, img2_shape, matches, keypoints_1, keypoints_2):
  x1 = [i for i in range(img1_shape[1])]
  y1 = [i for i in range(img1_shape[0])]
  x2 = [i for i in range(img2_shape[1])]
  y2 = [i for i in range(img2_shape[0])]
  (mean_x1, std_x1, mean_y1, std_y1) = [np.mean(x1), np.std(x1), np.mean(y1), np.std(y1)]
  (mean_x2, std_x2, mean_y2, std_y2) = [np.mean(x2), np.std(x2), np.mean(y2), np.std(y2)]
  T1 = np.array([[1/std_x1,     0   , -mean_x1/std_x1],
                [    0    , 1/std_y1, -mean_y1/std_y1],
                [    0    ,     0   ,        1       ]])
  T2 = np.array([[1/std_x2,     0   , -mean_x2/std_x2],
                [    0    , 1/std_y2, -mean_y2/std_y2],
                [    0    ,     0   ,        1       ]])
  
  normalized_coords1 = []
  normalized_coords2 = []
  for i in range(len(matches)):
    (temp1, temp2) = [list(keypoints_1[matches[i][0]].pt), list(keypoints_2[matches[i][1]].pt)]
    temp1.append(1)
    temp2.append(1)
    (temp1, temp2) = [np.array(temp1), np.array(temp2)]
    temp1 = T1 @ temp1
    temp2 = T2 @ temp2
    normalized_coords1.append(temp1)
    normalized_coords2.append(temp2)

  normalized_coords1 = np.array(normalized_coords1)
  normalized_coords2 = np.array(normalized_coords2)
  xy1_ = normalized_coords1[:, :-1]
  xy2_ = normalized_coords2[:, :-1]
  return xy1_, xy2_, T1, T2


def stitch_2_images(img1, img2, display):
  matches, keypoints_1, keypoints_2 = detect_keypoints_and_find_matches(img1, img2, display)
  xy1_, xy2_, T1, T2 = normalize__(img1.shape, img2.shape, matches, keypoints_1, keypoints_2)
  distance_threshold = 0.01 #0.01
  max_num_inliers = -1
  for _ in range(1000):
    rand_indices = random.sample(range(0, xy1_.shape[0]), 4)
    subset_xy2_ = xy2_[tuple([rand_indices])]
    subset_xy1_ = xy1_[tuple([rand_indices])]
    H_norm = est_homography(subset_xy1_, subset_xy2_)
    xy1_homo = apply_homography(H_norm, xy1_)
    diff = xy2_ - xy1_homo
    distances = np.linalg.norm(diff, axis=1)
    distances[distances<=distance_threshold] = -1
    distances[distances>distance_threshold] = 0
    num_inliers = np.sum(distances) / -1
    if max_num_inliers < num_inliers:
      max_num_inliers = num_inliers
      max_inlier_comb = tuple(np.arange(xy2_.shape[0])[distances==-1])

  print('Max number of inliers =', max_num_inliers)
  subset_xy1_ = xy1_[tuple([max_inlier_comb])]
  subset_xy2_ = xy2_[tuple([max_inlier_comb])]
  H_norm = est_homography(subset_xy1_, subset_xy2_)
  H = np.linalg.inv(T2) @ H_norm @ T1
  print('De-normalized Homography:\n', H)
  xy1 = np.mgrid[0:img1.shape[1]:0.25, 0:img1.shape[0]:0.25]
  xy1 = xy1.reshape((2, -1))
  xy1 = np.transpose(xy1)

  xy1_homo = apply_homography(H, xy1)
  combined_img = np.zeros((img1.shape[0], img1.shape[1]+img2.shape[1], 3)) # define the size of new canvas
  combined_img[:, -img2.shape[1]:, :] = img2
  counter_rows = 0
  counter_cols_L = 0
  counter_cols_R = 0
  counter_rows_cols = 0
  for i in range(xy1_homo.shape[0]):
    new_row = int(xy1_homo[i][1])
    new_col = int(xy1_homo[i][0]+img1.shape[1])
    if (new_row < 0) or (new_row > img1.shape[0]-1) or (new_col > img1.shape[1]+img2.shape[1]-1) or (new_col < 0):
      continue
    combined_img[new_row, new_col] = img1[int(xy1[i][1]), int(xy1[i][0])]
  combined_img = combined_img.astype(np.uint8)
  if display:
    cv2_imshow(combined_img)
  return combined_img


# img1 = cv2.imread('/content/tv/1.jpg')
# img2 = cv2.imread('/content/tv/2.jpg')
# img3 = cv2.imread('/content/tv/3.jpg')
img1 = cv2.imread('/content/hill/1.JPG')
img2 = cv2.imread('/content/hill/2.JPG')
img3 = cv2.imread('/content/hill/3.JPG')
stitched_12 = stitch_2_images(img1, img2, True)
stitched_123 = stitch_2_images(stitched_12, img3, True)

### Write-up (15 pt)
*  Describe how to remove incorrect matches with RANSAC 
*  Display the best homography H after RANSAC 
*  Display the blended images

- 1) RANSAC involves rejecting outliers. It does this by randomly sampling N samples from M (M>N) data points (here, match pairs) & fitting an equation to those N samples. Then, all the M data points are substituted in the above fitted equation & those data points which don't satisfy the equation upto a threshold distance are considered as incorrect matches (outliers) & they are removed.
  - This process is repeated for many iterations & the iteration which has the least number of outliers is selected to get the final estimate for the equation.

- 2) The following are the de-normalized Homography matrices for stitching (i) the 1st hill image with the 2nd hill image & (ii) the stitched 1st & 2nd hill image with the 3rd hill image:
  - <img src="https://drive.google.com/uc?id=1z0ipA7VsynFlsna9DXdj_0Q5Lor8tsRu" width="400"/>
  - <img src="https://drive.google.com/uc?id=1OAQtWGj9hDfdVMkJfm6a8GpRSPw4b-8V" width="400"/>

- 3) Blended images:
  - <img src="https://drive.google.com/uc?id=1zVbOac1xOpr8WYGUfwRDfqBL9ozW4KW5"/>
  - <img src="https://drive.google.com/uc?id=1qTLNd6yPPMlEbP5xhaOlQz01Tsr7zgGd"/>

# Part 3: Affine Structure from Motion (40 points)

## Overview
<img src="https://drive.google.com/uc?id=1nYd0eJjBtVIPuapfxuiVzswjswGN_Gq2&export=download" width="800"/>


This problem continues the interest point detection and tracking problem from HW2. Now, you will recover a 3D pointcloud from the image sequence hotel.seq0.png … hotel.seq50.png. You are encouraged to use your results from HW2, but in case you were not able to complete it, we have also included pre- computed intermediate results in the supplemental material. Submit your code so that we can reproduce your results.

The outline of the affine structure from motion algorithm:

<img src="https://drive.google.com/uc?id=1BSvHwRR5gNBwDGlrk-dcLCRcuIAvab__&export=download" width="700"/>


## Data

**WARNING: Colab deletes all files everytime runtime is disconnected. Make sure to re-download the inputs when it happens.**

In [2]:
# Download Data -- run this cell only one time per runtime
!gdown 1A0Rin_YMmWkExjI99vfLYvU_dy-9gFTT
!unzip "/content/Part2_data.zip" -d "/content/"
# Load Matches
data = loadmat('/content/Part2_data/tracks.mat')

Downloading...
From: https://drive.google.com/uc?id=1A0Rin_YMmWkExjI99vfLYvU_dy-9gFTT
To: /content/Part2_data.zip

  0% 0.00/5.44M [00:00<?, ?B/s]
100% 5.44M/5.44M [00:00<00:00, 254MB/s]
Archive:  /content/Part2_data.zip
   creating: /content/Part2_data/
   creating: /content/Part2_data/images/
  inflating: /content/Part2_data/images/hotel.seq0.png  
  inflating: /content/Part2_data/images/hotel.seq1.png  
  inflating: /content/Part2_data/images/hotel.seq10.png  
  inflating: /content/Part2_data/images/hotel.seq11.png  
  inflating: /content/Part2_data/images/hotel.seq12.png  
  inflating: /content/Part2_data/images/hotel.seq13.png  
  inflating: /content/Part2_data/images/hotel.seq14.png  
  inflating: /content/Part2_data/images/hotel.seq15.png  
  inflating: /content/Part2_data/images/hotel.seq16.png  
  inflating: /content/Part2_data/images/hotel.seq17.png  
  inflating: /content/Part2_data/images/hotel.seq18.png  
  inflating: /content/Part2_data/images/hotel.seq19.png  
  inflatin

## Code (20 pt)

In [ ]:
track_x = data['track_x']
track_y = data['track_y']

D = np.block([[data['track_x'][:, i], data['track_y'][:, i]] for i in range(data['track_y'].shape[1])]).reshape((102, -1))
# Remove the nan values (points which went out of frame)
nans = np.isnan(D)
keep_indices = []
for i in range(D.shape[1]):
  d_col = nans[:,i].tolist()
  if not any(d_col):
    keep_indices.append(i)
track_x = track_x[keep_indices, :]
track_y = track_y[keep_indices, :]

def affineSFM(x, y):
  '''
  Function: Affine structure from motion algorithm
  % Normalize x, y to zero mean
  % Create measurement matrix
  D = [xn' ; yn'];
  % Decompose and enforce rank 3
  % Apply orthographic constraints
  '''
  mean_x = np.mean(x, axis=0)
  mean_y = np.mean(y, axis=0)
  x -= mean_x
  y -= mean_y
  D = np.block([[x[:, i], y[:, i]] for i in range(y.shape[1])]).reshape((102, -1))
  U, S, VT = np.linalg.svd(D)
  U3 = U[:, :3]
  S3 = np.diag(S[:3])
  VT3 = VT[:3, :]
  S3_sqrt = sqrtm(S3)
  A_ = U3 @ S3_sqrt
  X_ = S3_sqrt @ VT3
  scalar_mat = []
  for i in range(0, A_.shape[0], 2):
    a1 =  A_[i]
    a2 = A_[i+1]
    block1 = (np.expand_dims(a1, axis=1) @ np.expand_dims(a1, axis=0)).reshape((1,9))
    block2 = (np.expand_dims(a2, axis=1) @ np.expand_dims(a2, axis=0)).reshape((1,9))
    block3 = (np.expand_dims(a1, axis=1) @ np.expand_dims(a2, axis=0)).reshape((1,9))
    if i==0:
      coeff_mat = block1
    else:
      coeff_mat = np.concatenate((coeff_mat, block1), axis=0)
    coeff_mat = np.concatenate((coeff_mat, block2), axis=0)
    coeff_mat = np.concatenate((coeff_mat, block3), axis=0)
    scalar_mat += [1,1,0]
  scalar_mat = np.array(scalar_mat)
  least_sq = np.linalg.lstsq(coeff_mat, scalar_mat, rcond=None)
  L = least_sq[0].reshape((3,3))
  w, v = np.linalg.eig(L)
  C = np.linalg.cholesky(L)
  A = A_ @ C
  X = np.linalg.inv(C) @ X_
  return A, X

def plot_structure(X):
  fig = go.Figure(data=[go.Scatter3d(x=X[0, :], y=X[1, :], z=X[2, :], mode='markers',
    marker=dict(size=3))])
  fig.show()

def plot_camera_motion(A):
  for i in range(0, A.shape[0], 2):
    position = np.cross(A[i], A[i+1])
    mag = np.linalg.norm(position)
    norm_position = position / mag
    norm_position = np.expand_dims(norm_position, axis=0)
    if i==0:
      cam_motion = norm_position
    else:
      cam_motion = np.concatenate((cam_motion, norm_position), axis=0)
  print(cam_motion.shape)
  fig = go.Figure(data=[go.Scatter3d(x=cam_motion[:, 0], y=cam_motion[:, 1], z=cam_motion[:, 2], mode='markers',
    marker=dict(size=3))])
  # fig = go.Figure()
  # fig.add_trace(go.Scatter(x=np.linspace(0, A.shape[0]/2, num=1+A.shape[0]//2), y=cam_motion[:, 0], mode='lines', name='x'))
  # fig.add_trace(go.Scatter(x=np.linspace(0, A.shape[0]/2, num=1+A.shape[0]//2), y=cam_motion[:, 1], mode='lines', name='y'))
  # fig.add_trace(go.Scatter(x=np.linspace(0, A.shape[0]/2, num=1+A.shape[0]//2), y=cam_motion[:, 2], mode='lines', name='z'))
  fig.show()

A, X = affineSFM(track_x, track_y)
print(A.shape, X.shape)
plot_structure(X)
plot_camera_motion(A)

### Write-up (20 pt)


*   Plot the predicted 3D locations of the tracked points for 3 different viewpoints. Choose the viewpoints so that the 3D structure is clearly visible.
*   Plot the predicted 3D path of the cameras. The camera position for each frame is given by the cross product a_k = a_i x a_j. Normalize a_k to be unit length for consistent results. Give 3 plots, one for each dimension of a_k 
<!-- We provide the function plotSfM.m for visualizing the recovered 3D shape and camera positions in each frame. -->


### Hint


- 3D Structure Visualizations:
  - <img src="https://drive.google.com/uc?id=1j9_aviLnIdeEPcDrt5CcmdNIi-gBj5KD"/>
  - <img src="https://drive.google.com/uc?id=1_3sXKJTJswiq4-jo1E2vkPIEP8gXakA0"/>
  - <img src="https://drive.google.com/uc?id=1En4G3LZ1NR61qXeWdZegyK2Hyf4t22zq"/>
  - <img src="https://drive.google.com/uc?id=185GUlxabZtI2lKH9OwXO5yBjLk1n73rx"/>

- Camera motion:
  - 3D plot:
    - <img src="https://drive.google.com/uc?id=1EtwMMDKceDsgh3S13ZKYWgEff3wl_HnF"/>
  - Separate 2D plots (x, y & z in order):
    - <img src="https://drive.google.com/uc?id=1BQ-b7zp9qM3w82oCNYbn3ya00Ox83EUJ"/>
    - <img src="https://drive.google.com/uc?id=1aqmDNXmDPBdsbwVTdLrwFaJShvOAEvys"/>
    - <img src="https://drive.google.com/uc?id=1oUZw8USY99joCoIB4prjl9CbkvW1YZAk"/>
    
<!--     

*   Reference: 
    - Tomasi and Kanade. Shape and Motion from Image Streams under Orthography: a Factorization Method. 1992 -->